# Real-time music auto-tagging
In this tutorial, we use Essentia's TensorFlow integration to perform auto-tagging in real-time.
Additionally, this serves as an example of TensorFlow inference in streaming mode and can be easily adapted to work offline.


## Setup
To install Essentia with TensorFlow support, refer to the [Setup](https://essentia.upf.edu/tutorial_tensorflow_auto-tagging_classification_embeddings.html#setup) section of our previous *Music auto-tagging, classification, and embedding extraction* tutorial for instructions.

Additionally, we rely on the `pysoundcard` package to capture the audio loopback of the system and feed Essentia in real-time. This way we can easily test our models with any music coming from our local player or browser.

In [1]:
#!pip -q install pysoundcard

Let's download `MusiCNN`,  one of our auto-tagging models. This and more models are available from the [Essentia models](https://essentia.upf.edu/models/)' site.

In [2]:
#!wget -q https://essentia.upf.edu/models/autotagging/msd/msd-musicnn-1.pb
#!wget -q https://essentia.upf.edu/models/autotagging/msd/msd-musicnn-1.json

Then we import the required packages and Essentia algorithms.
In this case, we use the TensorFlow functionalities in streaming mode.


In [3]:
import json

from essentia.streaming import (
    VectorInput,
    FrameCutter,
    TensorflowInputMusiCNN,
    VectorRealToTensor,
    TensorToPool,
    TensorflowPredict,
    PoolToTensor,
    TensorToVectorReal
)
from essentia import Pool, run, reset
from IPython import display
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax
import soundcard as sc

%matplotlib nbagg

2022-02-02 19:22:22.722249: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-02 19:22:22.722294: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-02-02 19:22:24.842606: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-02 19:22:24.842777: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-02 19:22:24.842789:

Define the analysis parameters.
To make this demo work in real-time, we tweaked some of the analysis parameters of `MusiCNN`.
While it was trained on patches of size 187 (\~3 seconds) we set `patch_size` to 64 (\~1 second) to increase the prediction rate.
You can experiment with the `patch_size` and `display_size` parameters to modify the prediction rate to your taste.

In [4]:
with open('msd-musicnn-1.json', 'r') as json_file:
    metadata = json.load(json_file)

model_file = 'msd-musicnn-1.pb'
input_layer = metadata['schema']['inputs'][0]['name']
output_layer = metadata['schema']['outputs'][0]['name']
classes = metadata['classes']
n_classes = len(classes)

# Analysis parameters.
sample_rate = 16000
frame_size = 512 
hop_size = 256
n_bands = 96
patch_size = 64
display_size = 10

buffer_size = patch_size * hop_size

Instantiate the algorithms. With this, we create a network similar to the one used inside `TensorflowPredictMusiCNN`, the wrapper algorithm presented in the previous tutorial. However, by instantiating the algorithms separately we gain additional control required for real-time usage.

In [5]:
buffer = np.zeros(buffer_size, dtype='float32')
vimp = VectorInput(buffer)
fc = FrameCutter(frameSize=frame_size, hopSize=hop_size)
tim = TensorflowInputMusiCNN()
vtt = VectorRealToTensor(shape=[1, 1, patch_size, n_bands],
                         lastPatchMode='discard')
ttp = TensorToPool(namespace=input_layer)
tfp = TensorflowPredict(graphFilename=model_file,
                        inputs=[input_layer],
                        outputs=[output_layer])
ptt = PoolToTensor(namespace=output_layer)
ttv = TensorToVectorReal()
pool = Pool()

[   INFO   ] Successfully loaded graph file: `msd-musicnn-1.pb`


Connect the algorithms. We also store the mel-spectrograms in the `Pool` for visualization purposes.

In [6]:
vimp.data   >> fc.signal
fc.frame    >> tim.frame
tim.bands   >> vtt.frame
tim.bands   >> (pool, 'melbands')
vtt.tensor  >> ttp.tensor
ttp.pool    >> tfp.poolIn
tfp.poolOut >> ptt.pool
ptt.tensor  >> ttv.tensor
ttv.frame   >> (pool, output_layer)

Initialize the plots and start processing the loopback stream.

In [7]:
def callback_console(data):
    buffer[:] = data.flatten()

    # Generate predictions.
    reset(vimp)
    run(vimp)
    
    act_buffer[:, -1] =print(classes[np.argmax(softmax(20 * pool[output_layer][-1, :].T))])


In [8]:
act_buffer = np.zeros([n_classes, display_size])

# Capture and process the speakers loopback.
with sc.all_microphones(include_loopback=True)[0].recorder(samplerate=sample_rate) as mic:
    while True:
        callback_console(mic.record(numframes=buffer_size).mean(axis=1))

[   INFO   ] On connection TensorflowInputMusiCNN::bands → VectorRealToTensor::frame:
[   INFO   ] BUFFER SIZE MISMATCH: max=0 - asked for read size 64
[   INFO   ] resizing buffer to 560/70
2022-02-02 19:22:29.697988: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3194040000 Hz


electronic
electronic
electronic
indie
blues
rock
blues
indie
rock
indie
rock
rock
electronic


KeyboardInterrupt: 